## 1️⃣ GPU Kontrolü

In [ ]:
# GPU kontrolü
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
if 'Tesla T4' in result.stdout or 'GPU' in result.stdout:
    print("✅ GPU aktif!")
    print(result.stdout[:500])
else:
    print("⚠️ GPU bulunamadı! Runtime → Change runtime type → T4 GPU seçin")

## 2️⃣ Bağımlılıkları Kur

In [ ]:
# Gerekli paketleri kur
!pip install -q fastapi uvicorn python-multipart pyngrok ultralytics opencv-python-headless pillow nest-asyncio
print("✅ Paketler kuruldu!")

## 3️⃣ ngrok Token Ayarla

1. https://dashboard.ngrok.com/get-started/your-authtoken adresine git
2. Ücretsiz hesap oluştur
3. Token'ı kopyala ve aşağıya yapıştır

In [ ]:
# ngrok token'ınızı buraya yapıştırın
NGROK_TOKEN = ""  # @param {type:"string"}

if not NGROK_TOKEN:
    print("⚠️ ngrok token gerekli!")
    print("1. https://dashboard.ngrok.com/get-started/your-authtoken adresine git")
    print("2. Token'ı kopyala ve yukarıdaki NGROK_TOKEN değişkenine yapıştır")
else:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ ngrok token ayarlandı!")

## 4️⃣ API Sunucusu Kodu

In [ ]:
%%writefile app.py
"""
Teknova AI Animal Tracking API
Google Colab + Ücretsiz GPU
"""

import io
import time
from typing import List, Dict, Optional, Any
from collections import defaultdict

import numpy as np
import cv2
from PIL import Image
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from ultralytics import YOLO

# ===========================================
# Configuration
# ===========================================

ANIMAL_CLASS_IDS = {14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
ANIMAL_NAMES = {
    14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse',
    18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear',
    22: 'zebra', 23: 'giraffe'
}
TURKISH_PREFIXES = {
    'cow': 'INEK', 'sheep': 'KOYUN', 'goat': 'KECI',
    'horse': 'AT', 'dog': 'KOPEK', 'cat': 'KEDI',
    'bird': 'KUS', 'elephant': 'FIL', 'bear': 'AYI',
    'zebra': 'ZEBRA', 'giraffe': 'ZURAFA'
}

SIMILARITY_THRESHOLD = 0.92
CONFIDENCE_THRESHOLD = 0.4

# ===========================================
# Models
# ===========================================

class DetectedAnimal(BaseModel):
    track_id: int
    animal_id: str
    class_name: str
    bbox: List[int]
    confidence: float
    re_id_confidence: float
    is_identified: bool
    is_new: bool
    velocity: List[float] = [0.0, 0.0]
    direction: float = 0.0
    health_score: Optional[float] = None
    behavior: Optional[str] = None

class ProcessResult(BaseModel):
    frame_id: int
    timestamp: float
    fps: float
    animal_count: int
    total_registered: int
    new_this_frame: int
    animals: List[DetectedAnimal]
    frame_size: List[int]

# ===========================================
# Feature Extractor
# ===========================================

class FeatureExtractor:
    def __init__(self):
        self.size = (128, 256)
        self.bins = 32
    
    def extract(self, image: np.ndarray, bbox: tuple) -> Optional[np.ndarray]:
        x1, y1, x2, y2 = [int(v) for v in bbox]
        h, w = image.shape[:2]
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        
        if x2 <= x1 or y2 <= y1:
            return None
        
        crop = image[y1:y2, x1:x2]
        if crop.size == 0:
            return None
        
        try:
            crop = cv2.resize(crop, self.size)
        except:
            return None
        
        features = []
        
        # HSV histogram
        hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
        for i, bins in enumerate([self.bins, self.bins, self.bins//2]):
            hist = cv2.calcHist([hsv], [i], None, [bins], [0, 256 if i > 0 else 180])
            cv2.normalize(hist, hist)
            features.append(hist.flatten())
        
        # Regional (top, mid, bottom)
        h_crop = crop.shape[0]
        for region in [crop[:h_crop//3], crop[h_crop//3:2*h_crop//3], crop[2*h_crop//3:]]:
            hsv_r = cv2.cvtColor(region, cv2.COLOR_BGR2HSV)
            hist = cv2.calcHist([hsv_r], [0], None, [self.bins], [0, 180])
            cv2.normalize(hist, hist)
            features.append(hist.flatten())
        
        # Edge
        gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150)
        edge_hist = cv2.calcHist([edges], [0], None, [16], [0, 256])
        cv2.normalize(edge_hist, edge_hist)
        features.append(edge_hist.flatten())
        
        combined = np.concatenate(features)
        norm = np.linalg.norm(combined)
        if norm > 0:
            combined = combined / norm
        
        return combined.astype(np.float32)

# ===========================================
# Gallery
# ===========================================

class Gallery:
    def __init__(self):
        self.records = {}
        self.features = {}
        self.id_counters = defaultdict(int)
        self.class_index = defaultdict(list)
    
    def generate_id(self, class_name: str) -> str:
        prefix = TURKISH_PREFIXES.get(class_name.lower(), class_name.upper()[:4])
        self.id_counters[prefix] += 1
        return f"{prefix}_{self.id_counters[prefix]:04d}"
    
    def register(self, features: np.ndarray, class_name: str, confidence: float) -> str:
        animal_id = self.generate_id(class_name)
        self.records[animal_id] = {
            "animal_id": animal_id,
            "class_name": class_name,
            "first_seen": time.time(),
            "last_seen": time.time(),
            "total_detections": 1,
            "best_confidence": confidence
        }
        self.features[animal_id] = features.copy()
        self.class_index[class_name].append(animal_id)
        print(f"🆕 Yeni hayvan: {animal_id}")
        return animal_id
    
    def search(self, query: np.ndarray, class_name: str, top_k: int = 5):
        if class_name not in self.class_index:
            return []
        
        results = []
        query_norm = query / (np.linalg.norm(query) + 1e-8)
        
        for aid in self.class_index[class_name]:
            if aid not in self.features:
                continue
            feat = self.features[aid]
            feat_norm = feat / (np.linalg.norm(feat) + 1e-8)
            sim = float(np.dot(query_norm, feat_norm))
            results.append((aid, sim))
        
        results.sort(key=lambda x: x[1], reverse=True)
        return results[:top_k]
    
    def update(self, animal_id: str, features: np.ndarray):
        if animal_id in self.features:
            alpha = 0.02
            self.features[animal_id] = (1 - alpha) * self.features[animal_id] + alpha * features
            self.records[animal_id]["last_seen"] = time.time()
            self.records[animal_id]["total_detections"] += 1
    
    def reset(self):
        self.records.clear()
        self.features.clear()
        self.id_counters.clear()
        self.class_index.clear()
        print("🗑️ Galeri sıfırlandı")
    
    @property
    def size(self):
        return len(self.records)

# ===========================================
# Detector
# ===========================================

class Detector:
    def __init__(self):
        print("🔄 YOLOv8 yükleniyor...")
        self.model = YOLO('yolov8n.pt')
        print("✅ Model hazır!")
        self.extractor = FeatureExtractor()
        self.gallery = Gallery()
        self.frame_count = 0
        self.track_id = 0
    
    def process(self, image: np.ndarray) -> ProcessResult:
        start = time.time()
        self.frame_count += 1
        h, w = image.shape[:2]
        
        animals = []
        new_count = 0
        used_ids = set()
        
        # YOLO detection
        results = self.model(image, verbose=False, conf=CONFIDENCE_THRESHOLD)
        
        detections = []
        for result in results:
            if result.boxes is None:
                continue
            for box in result.boxes:
                cls_id = int(box.cls[0].item())
                if cls_id not in ANIMAL_CLASS_IDS:
                    continue
                
                class_name = ANIMAL_NAMES.get(cls_id, 'unknown')
                bbox = [int(v) for v in box.xyxy[0].cpu().numpy()]
                conf = float(box.conf[0].item())
                
                features = self.extractor.extract(image, bbox)
                if features is None:
                    continue
                
                detections.append({
                    "bbox": bbox,
                    "confidence": conf,
                    "class_name": class_name,
                    "features": features
                })
        
        # Sort by confidence
        detections.sort(key=lambda x: x["confidence"], reverse=True)
        
        for det in detections:
            self.track_id += 1
            
            # Search gallery
            matches = self.gallery.search(det["features"], det["class_name"])
            
            best_match = None
            for aid, sim in matches:
                if aid not in used_ids and sim >= SIMILARITY_THRESHOLD:
                    best_match = (aid, sim)
                    break
            
            if best_match:
                animal_id, similarity = best_match
                used_ids.add(animal_id)
                self.gallery.update(animal_id, det["features"])
                is_new = False
            else:
                animal_id = self.gallery.register(
                    det["features"],
                    det["class_name"],
                    det["confidence"]
                )
                used_ids.add(animal_id)
                similarity = 1.0
                is_new = True
                new_count += 1
            
            animals.append(DetectedAnimal(
                track_id=self.track_id,
                animal_id=animal_id,
                class_name=det["class_name"],
                bbox=det["bbox"],
                confidence=det["confidence"],
                re_id_confidence=similarity,
                is_identified=True,
                is_new=is_new
            ))
        
        elapsed = time.time() - start
        fps = 1.0 / elapsed if elapsed > 0 else 0
        
        return ProcessResult(
            frame_id=self.frame_count,
            timestamp=time.time(),
            fps=fps,
            animal_count=len(animals),
            total_registered=self.gallery.size,
            new_this_frame=new_count,
            animals=animals,
            frame_size=[w, h]
        )

# ===========================================
# FastAPI App
# ===========================================

app = FastAPI(title="Teknova AI Animal Tracking")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

detector = None

@app.on_event("startup")
async def startup():
    global detector
    detector = Detector()

@app.get("/")
async def root():
    return {"name": "Teknova AI Animal Tracking", "status": "running"}

@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "gpu": "T4",
        "gallery_size": detector.gallery.size if detector else 0
    }

@app.post("/api/v1/detection/process-frame")
async def process_frame(file: UploadFile = File(...)):
    if detector is None:
        raise HTTPException(503, "Detector not ready")
    
    contents = await file.read()
    nparr = np.frombuffer(contents, np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    if image is None:
        raise HTTPException(400, "Invalid image")
    
    result = detector.process(image)
    return result.model_dump()

@app.get("/api/v1/detection/gallery")
async def get_gallery():
    if detector is None:
        raise HTTPException(503, "Detector not ready")
    
    return {
        "total": detector.gallery.size,
        "animals": list(detector.gallery.records.values()),
        "by_class": {k: len(v) for k, v in detector.gallery.class_index.items()}
    }

@app.post("/api/v1/detection/reset")
async def reset_gallery():
    if detector is None:
        raise HTTPException(503, "Detector not ready")
    
    detector.gallery.reset()
    return {"status": "success"}

## 5️⃣ Sunucuyu Başlat 🚀

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from pyngrok import ngrok
import uvicorn
import threading

# ngrok tunnel başlat
public_url = ngrok.connect(8000)
print("="*60)
print("🎉 SUNUCU HAZIR!")
print("="*60)
print(f"\n📱 iOS Uygulamasına Bu URL'i Yapıştır:\n")
print(f"   {public_url}")
print(f"\n" + "="*60)
print("\n⚠️ Bu pencereyi AÇIK TUTUN - kapatırsanız sunucu durur!")
print("\n🔄 Test için:")
print(f"   curl {public_url}/health")
print("="*60)

# Sunucuyu başlat
uvicorn.run("app:app", host="0.0.0.0", port=8000)